## 双均线策略示例

In [2]:
import pandas as pd
import numpy as np
import tushare as ts
import matplotlib.pyplot as plt


# 将日期作为索引，并修改收盘价为股票名
zgpa = ts.get_k_data('601318', start='2012-01-01', end='2014-12-31')
zgpa = zgpa.set_index('date')

# 将不需要的因素去除
zgpa = zgpa[['open', 'close', 'high', 'low']]
zgpa['mean_5'] = zgpa['close'].rolling(window=5).mean()
zgpa['mean_20'] = zgpa['close'].rolling(window=20).mean()
zgpa[['mean_5','mean_20']].plot()
plt.legend(loc='best')
plt.show()

<Figure size 640x480 with 1 Axes>

找出金叉和死叉的时间

In [5]:
# 在计算金叉、死叉时使用了一个比较精巧的方法,由F->T表示金叉，死叉类似
# zgpa['mean_5'] <= zgpa['mean_20']   FFFFFFFTTTTTTTTFFFFFF
# zgpa['mean_5'] > zgpa['mean_20']     TTTTTTTFFFFFFFFTTTTTT

zgpa = zgpa.dropna()
ser1 = zgpa['mean_5'] <= zgpa['mean_20']
ser2 = zgpa['mean_5'] > zgpa['mean_20']

golden_cross = zgpa[ser1 & ser2.shift(1)]
death_cross = zgpa[~(ser1 | ser2.shift(1))]

,open,close,high,low,mean_5,mean_20
date,,,,,,
2012-02-07,17.493,17.311,17.591,17.128,17.4156,16.50700
2012-04-13,17.449,17.684,17.818,17.440,17.1228,17.08280
2012-06-01,18.641,18.761,18.903,18.632,18.6498,18.55795
2012-06-13,18.917,20.145,20.225,18.917,18.8892,18.62465
2012-07-23,20.294,20.039,20.294,19.909,20.1588,20.09950
2012-09-10,18.835,18.777,18.952,18.612,18.1212,18.07010
2012-09-27,18.178,18.699,18.964,18.178,18.1432,18.08675
2012-12-06,17.455,17.468,17.661,17.334,16.9132,16.66025
2013-04-19,18.995,19.265,19.354,18.995,18.7714,18.68190


按照简单的金叉和死叉策略进行买卖

In [29]:
# 重新构建一个series,用来将金叉和死叉合并在一起
s1 = pd.Series(1, index = golden_cross.index)
s2 = pd.Series(0, index = death_cross.index) 
s1 = s1.append(s2).sort_index()

init_money = 10000
hands = 0
current_money = init_money
for i in range(len(s1)):
    price = zgpa['close'][s1.index[i]]
    # 金叉
    if s1.iloc[i] == 1:
        hands = current_money // (price*100)
        current_money -= hands *100 * price
    # 死叉
    else:
        current_money += hands * price * 100
        hands = 0

price = zgpa['close'][-1]
current_money += hands * price * 100

print(current_money - init_money)


date
2012-02-07    0
2012-03-07    1
2012-04-13    0
2012-05-28    1
2012-06-01    0
2012-06-05    1
2012-06-13    0
2012-07-16    1
2012-07-23    0
2012-07-26    1
2012-09-10    0
2012-09-21    1
2012-09-27    0
2012-10-26    1
2012-12-06    0
2013-02-22    1
2013-04-19    0
2013-04-25    1
2013-05-08    0
2013-05-13    1
2013-05-22    0
2013-05-27    1
2013-07-16    0
2013-07-18    1
2013-08-12    0
2013-09-30    1
2013-10-30    0
2013-12-17    1
2014-01-17    0
2014-01-28    1
2014-03-26    0
2014-04-24    1
2014-05-13    0
2014-05-21    1
2014-05-28    0
2014-06-24    1
2014-07-07    0
2014-08-20    1
2014-09-04    0
2014-09-16    1
2014-10-31    0
dtype: int64